<a href="https://colab.research.google.com/github/jcandane/stlfilereader/blob/main/STLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STL File to Complex

Given an STL File, obtain $\Omega_0$ (0-simplices) and $\Omega_2$ (2-simplices).

In [1]:
!pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
from stl import mesh

def ssort(A, return_unique=False):

    ARG = np.argsort(A[0] , kind="mergesort")
    A   = A[:,ARG]
    
    ikk = np.array([], dtype=np.int32)
    for k in range(1,len(A)):
        #ik = np.where(mydiff(A[k-1]) != 0)[0]
        ik = np.where( np.diff(A[k-1], prepend=A[k-1,0]-1, append=A[k-1,-1]+1) != 0)[0]
        ik = np.union1d(ikk, ik)
        for l in range(len(ik)-1):
            argssort = np.argsort(A[k,ik[l]:ik[l+1]] , kind="mergesort")
            A[:,ik[l]:ik[l+1]] = (A[:,ik[l]:ik[l+1]])[:,argssort]
            ARG[ik[l]:ik[l+1]] = (ARG[ik[l]:ik[l+1]])[argssort]
        ikk = 1*ik

    k = len(A)-1 ## 8/30
    if return_unique:
        #ik = np.where(mydiff(A[k-1]) != 0)[0]
        ik = np.where( np.diff(A[k], prepend=A[k,0]-1, append=A[k,-1]+1) != 0)[0]
        ik = np.union1d(ikk, ik)
        return A, ARG, ik[:-1]
        ### !!! the final ikk keeps track of the uniques???
    return A

def bsearch_leftmost(A, T, L=0, R=None):
    if R is None:
        R = len(A)
    while L < R:
        m = (L + R) // 2 ##int( np.floor((L + R) / 2) )
        if A[m] < T:
            L = m + 1
        else:
            R = m
    return L

def bsearch_rightmost(A, T, L=0, R=None):
    if R is None:
        R = len(A)
    while L < R:
        m = (L + R) // 2 #int(np.floor((L + R) / 2)) # #
        if A[m] > T:
            R = m
        else:
            L = m + 1
    return R - 1

def binary_search_interval(A, value, L=0, R=None):

    L, R = bsearch_leftmost(A, value, L=L, R=R), bsearch_rightmost(A, value, L=L, R=R)

    if L <= R:
        return L, R+1
    else:
        return L, L

def tuplebsearch_interval(B, value, L=0, R=None): ### (value, B, L=0, R=None)
    for columns in range(B.shape[0]): ### over entries/columns of the tuple
        L, R = binary_search_interval(B[columns,:], value[columns], L=L, R=R)
    return L, R #np.array([L,R]) #np.array

In [3]:
your_mesh = mesh.Mesh.from_file('gear.stl') ## https://www.amtekcompany.com/teaching-resources/stl-files/

R_Ixy = your_mesh.vectors
R_Ix  = R_Ixy.reshape( (R_Ixy.shape[0]*3, 3) , order="C") ### get list-of-points per triangule -> list-of-points (many dupes)

Ω0, iR, uR = ssort( R_Ix.T , return_unique=True)
Ω0         = Ω0[:,uR] #### R is the location of the all unique points

######
Ω2 = np.zeros( (R_Ixy.shape[0], 3) , dtype=np.int32)
for i in range(Ω2.shape[0]):
    Ω2[i,0] = tuplebsearch_interval(Ω0, R_Ixy[i,0])[0]
    Ω2[i,1] = tuplebsearch_interval(Ω0, R_Ixy[i,1])[0]
    Ω2[i,2] = tuplebsearch_interval(Ω0, R_Ixy[i,2])[0]
Ω2 = Ω2.swapaxes(0,1)

print( Ω2.shape, Ω0[:,Ω2].shape )
print( np.allclose( Ω0[:,Ω2].swapaxes(0,2), R_Ixy ) )

(3, 4748) (3, 3, 4748)
True


## Let's compute the edges $\Omega_1$

In [4]:
Ω1 = ( np.concatenate( (Ω2[:-1,:], np.roll( Ω2, 1, axis=0)[:-1,:], np.roll( Ω2, 2, axis=0)[:-1,:] ) , axis=1) ) ### there are duplicates
### for every 2-tuple (A, B) -> (B, A)

Ω1, B, u1 = ssort( Ω1 , return_unique=True)
Ω1        = Ω1[:,u1]

## Let's Compute the Euler Characteristic of the STL object

In [5]:
χ = Ω0.shape[1] - Ω1.shape[1]/2 + Ω2.shape[1] ## Euler Characteristic (E/2, because it is double counted...)